# Create an interface with no strain matching

<h2 style="color:green">Usage</h2>

1. Make sure to select Input Materials (in the outer runtime) before running the notebook.
1. Set notebook parameters in cell 1.1. below (or use the default values).
1. Set slab parameters for the substrate and film in cell 2.1. (or use default).
1. Set interface parameters in cell 3.1. (or use default).
1. Click “Run” > “Run All” to run all cells. 
1. Scroll down to view results. 

## Summary
1. Prepare the Environment: Set up the notebook and install packages, preview the input materials
1. Create substrate and film slabs and select the terminations
1. Generate an interface by wrapping the film slab into the substrate slab. Optionally, the film slab can be scaled to match the substrate lattice.
1. Select the interface with the desired strain and visualize it

## Notes
1. For more information, see [Introduction](Introduction.ipynb)
<!-- # TODO: use a hashtag-based anchor link to interface creation documention above -->


## 1. Prepare Environment
### 1.1. Set notebook parameters

In [ ]:
# Create slabs for the film and substrate when creating the interface, otherwise use provided materials directly.
CREATE_SLABS = True 

FILM_INDEX = 1
FILM_MILLER_INDICES = (0, 0, 1)
FILM_THICKNESS = 1  # in atomic layers
FILM_VACUUM = 0.0  # in angstroms
FILM_TERMINATION_FORMULA = None  # if None, the first termination will be used
FILM_XY_SUPERCELL_MATRIX = [[1, 0], [0, 1]]
FILM_USE_ORTHOGONAL_C = True
FILM_XY_SHIFT = [0.0, 0.0]  # in Angstroms, shift the film slab in the XY plane relative to the substrate slab

SUBSTRATE_INDEX = 0
SUBSTRATE_MILLER_INDICES = (1, 1, 1)
SUBSTRATE_THICKNESS = 3  # in atomic layers
SUBSTRATE_VACUUM = 3.0  # in angstroms
SUBSTRATE_TERMINATION_FORMULA = None  # if None, the first termination will be used
SUBSTRATE_XY_SUPERCELL_MATRIX = [[1, 0], [0, 1]]
SUBSTRATE_USE_ORTHOGONAL_C = True

# Whether to convert materials to conventional cells before creating slabs.
USE_CONVENTIONAL_CELL = False

INTERFACE_DISTANCE = 3.0  # in Angstrom
INTERFACE_VACUUM = 20.0  # in Angstrom


### 1.2. Install Packages
Note: the step executes only when using Pyodide. For other environments (e.g. Colab, JupyterLab), the packages should be installed via `pip install .` (see [README](../../README.ipynb)).

In [ ]:
import sys

if sys.platform == "emscripten":
    import micropip
    await micropip.install("mat3ra-api-examples", deps=False)
    await micropip.install('mat3ra-utils')
    from mat3ra.utils.jupyterlite.packages import install_packages
    await install_packages("create_interface_with_min_strain_zsl.ipynb")

### 1.3. Get input materials and assign `substrate` and `film`
Materials are loaded with `get_data()`. The first material is assigned as substrate and the second as film.

In [ ]:
from utils.jupyterlite import get_materials

materials = get_materials(globals())
substrate = materials[SUBSTRATE_INDEX]
try: 
    film = materials[FILM_INDEX]
except IndexError:
    print("Please select a film material. Film is set to substrate.")
    film = substrate

### 1.4. Preview Substrate and Film

In [ ]:
from utils.visualize import visualize_materials as visualize
visualize([substrate, film], repetitions=[3, 3, 1], rotation="0x")

## 2. Create slabs for the interface



### 2.2. Get possible terminations for the slabs

In [ ]:
from mat3ra.made.tools.helpers import get_slab_terminations

film_slab_terminations = get_slab_terminations(film, miller_indices=FILM_MILLER_INDICES)
substrate_slab_terminations = get_slab_terminations(substrate, miller_indices=SUBSTRATE_MILLER_INDICES)

### 2.3. Visualize slabs for all possible terminations

In [ ]:
from mat3ra.made.tools.helpers import create_slab
from mat3ra.made.tools.helpers import select_slab_termination

film_slabs = [create_slab(film, miller_indices=FILM_MILLER_INDICES, termination_top=termination, vacuum=0) for termination
              in
              film_slab_terminations]

substrate_slabs = [create_slab(substrate, miller_indices=SUBSTRATE_MILLER_INDICES, termination_top=termination, vacuum=0)
                   for termination in
                   substrate_slab_terminations]

film_slabs_with_titles = [{"material": slab, "title": str(termination)} for slab, termination in
                          zip(film_slabs, film_slab_terminations)]
substrate_slabs_with_titles = [{"material": slab, "title": str(termination)} for slab, termination in
                               zip(substrate_slabs, substrate_slab_terminations)]

visualize(film_slabs_with_titles, repetitions=[3, 3, 1], rotation="-90x")
visualize(substrate_slabs_with_titles, repetitions=[3, 3, 1], rotation="-90x")

### 2.3. Select terminations for the Slabs

In [ ]:
film_termination = select_slab_termination(film_slab_terminations, FILM_TERMINATION_FORMULA)
substrate_termination = select_slab_termination(substrate_slab_terminations, SUBSTRATE_TERMINATION_FORMULA)

### 2.1. Create Substrate and Layer Slabs
Slab Configuration lets define the slab thickness, vacuum, and the Miller indices of the interfacial plane and get the slabs with possible terminations.
Define the substrate slab cell that will be used as a base for the interface and the film slab cell that will be placed on top of the substrate slab.

In [ ]:
from mat3ra.made.tools.helpers import create_slab

if CREATE_SLABS:
    film_slab = create_slab(
        crystal=film,
        miller_indices=FILM_MILLER_INDICES,
        termination_bottom=film_termination,
        number_of_layers=FILM_THICKNESS, # in atomic layers
        vacuum=FILM_VACUUM, # in angstroms
        xy_supercell_matrix=FILM_XY_SUPERCELL_MATRIX,
        use_orthogonal_c=FILM_USE_ORTHOGONAL_C,
        use_conventional_cell=USE_CONVENTIONAL_CELL
    )

    substrate_slab = create_slab(
        crystal=substrate,
        miller_indices=SUBSTRATE_MILLER_INDICES,
        termination_top=substrate_termination,
        number_of_layers=SUBSTRATE_THICKNESS, # in atomic layers
        vacuum=SUBSTRATE_VACUUM, # in angstroms
        xy_supercell_matrix=SUBSTRATE_XY_SUPERCELL_MATRIX,
        use_orthogonal_c=SUBSTRATE_USE_ORTHOGONAL_C,
        use_conventional_cell=USE_CONVENTIONAL_CELL
    )
else:
    film_slab = film
    substrate_slab = substrate


## 3. Create an interface
### 3.1. Create an interface with no strain matching
We use no strain matching for the interface construction. Substrate slab cell is used as a base and the film slab is placed on top of it with the specified distance.

In [ ]:
from mat3ra.made.tools.helpers import create_interface_simple_between_slabs as create_simple_interface_between_slabs

interface = create_simple_interface_between_slabs(
    film_slab=film_slab,
    substrate_slab=substrate_slab,
    gap =INTERFACE_DISTANCE,
    vacuum=INTERFACE_VACUUM,
    xy_shift=FILM_XY_SHIFT
)

In [ ]:
visualize([interface], repetitions=[3, 3, 1])
visualize([interface], repetitions=[3, 3, 1], rotation="-90x")

## 5. Pass data to the outside runtime

In [ ]:
from utils.jupyterlite import set_materials
set_materials(interface)